In [1]:
!pip install tensorflow keras scikit-learn nltk pandas numpy

In [3]:
import pandas as pd
import numpy as np
import nltk
import re
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords

# Download stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

Matplotlib created a temporary cache directory at C:\Users\nazla\AppData\Local\Temp\matplotlib-66c2excm because the default path (C:\Users\nazla\.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Matplotlib is building the font cache; this may take a moment.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nazla\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Load dataset
df = pd.read_csv("C:/Users/nazla/Documents/A. College Tasks 3rd Year/A. International Research/datasets/text_emotion.csv")

# Select relevant columns for each task
df = df[['content', 'sentiment']]
df.head()

,content,sentiment
0,@tiffanylue i know i was listenin to bad habi...,empty
1,Layin n bed with a headache ughhhh...waitin o...,sadness
2,Funeral ceremony...gloomy friday...,sadness
3,wants to hang out with friends SOON!,enthusiasm
4,@dannycastillo We want to trade with someone w...,neutral


In [5]:
# Function to clean text
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'\@\w+|\#', '', text)  # Remove mentions and hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuations
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

df['content'] = df['content'].apply(clean_text)

# Encode sentiment labels
sentiment_label_encoder = LabelEncoder()
df['sentiment_label'] = sentiment_label_encoder.fit_transform(df['sentiment'])

In [6]:
# Save label encoders for deployment
with open("sentiment_label_encoder.pkl", "wb") as f:
    pickle.dump(sentiment_label_encoder, f)

In [7]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])

In [8]:
# Save tokenizer for deployment
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

In [9]:
# Convert text to sequences and pad them
sequences = tokenizer.texts_to_sequences(df['content'])
max_length = 50  # Max sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

In [10]:
# Splitting data for Sentiment Model
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(
    padded_sequences, df['sentiment_label'], test_size=0.2, random_state=42, stratify=df['sentiment_label']
)

# Convert labels to categorical
num_classes_s = len(sentiment_label_encoder.classes_)
y_train_s = tf.keras.utils.to_categorical(y_train_s, num_classes_s)
y_test_s = tf.keras.utils.to_categorical(y_test_s, num_classes_s)

In [11]:
# Function to create LSTM model
def create_lstm_model(output_classes):
    model = Sequential([
        Embedding(input_dim=10000, output_dim=128, input_length=max_length),
        LSTM(128, return_sequences=True),
        Dropout(0.2),
        LSTM(64),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(output_classes, activation='softmax')  # Multi-class classification
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [15]:
# Train Sentiment Analysis Model
sentiment_model = create_lstm_model(num_classes_s)
sentiment_model.summary()
sentiment_model.fit(X_train_s, y_train_s, epochs=10, batch_size=32, validation_data=(X_test_s, y_test_s))

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 375s 338ms/step - accuracy: 0.2028 - loss: 2.2005 - val_accuracy: 0.2160 - val_loss: 2.1490
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 420s 420ms/step - accuracy: 0.2117 - loss: 2.1548 - val_accuracy: 0.2160 - val_loss: 2.1475
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 209s 205ms/step - accuracy: 0.2128 - loss: 2.1537 - val_accuracy: 0.2160 - val_loss: 2.1516
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 277s 277ms/step - accuracy: 0.2148 - loss: 2.1476 - val_accuracy: 0.2160 - val_loss: 2.1484
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 308s 262ms/step - accuracy: 0.2153 - loss: 2.1462 - val_accuracy: 0.2160 - val_loss: 2.1483
Epoch 9/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 333s 333ms/step - accuracy: 0.2145 - loss: 2.1484 - val_accuracy: 0.2115 - val_loss: 2.1472
Epoch 10/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 470s 420ms/step - accuracy: 0.2115 - loss: 2.1548 - val_accuracy: 0.2160 - val_loss: 2.1459


In [16]:
# Save Model
model.save("sentiment_lstm_model.h5")
print("Model, tokenizer, and label encoder saved successfully!")

NameError: name 'model' is not defined